# HullBuilder testings

## Imports and setup

In [1]:
from pcgsepy.common.api_call import get_base_values, GameMode, toggle_gamemode
from pcgsepy.common.vecs import Vec, Orientation
from pcgsepy.lsystem.structure_maker import LLStructureMaker
from pcgsepy.setup_utils import setup_matplotlib, get_default_lsystem
from pcgsepy.structure import Structure, place_structure, place_blocks

In [2]:
setup_matplotlib(larger_fonts=False)

used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerSquareInverted',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerSquare',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

lsystem = get_default_lsystem(used_ll_blocks=used_ll_blocks)

In [3]:
import numpy as np
import plotly.express as px

def interactive_plot(structure: Structure,
                     title: str) -> None:
    content = structure.as_grid_array()
    arr = np.nonzero(content)
    x, y, z = arr
    cs = [content[i, j, k] for i, j, k in zip(x, y, z)]
    ss = [structure._clean_label(structure.ks[v - 1]) for v in cs]
    fig = px.scatter_3d(x=x,
                        y=y,
                        z=z,
                        color=ss,
                        # color_discrete_map=_get_colour_mapping(ss),
                        labels={
                            'x': 'x',
                            'y': 'y',
                            'z': 'z',
                            'color': 'Block type'
                        },
                        title=title)

    fig.update_traces(marker=dict(size=4,
                                line=dict(width=3,
                                        color='DarkSlateGrey')),
                        selector=dict(mode='markers'))

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=2, y=2, z=2)
        )

    fig.update_layout(scene=dict(aspectmode='data'),
                        scene_camera=camera,
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')

    fig.show()

## Spaceship string

Define here the high-level spaceship string.

In [4]:
spaceship_string = 'cockpitcorridorsimple(1)corridorgyros(1)[RotYcwXcorridorsimple(1)corridorcargo(1)]corridorsimple(1)corridorreactors(1)corridorsimple(1)thrusters'
# spaceship_string = 'corridorgyros(2)[RotYccwXcorridorsimple(1)][RotYcwZcorridorsimple(1)]thrusters' # <- YccwXYcwZintersection seems to be wrong

## Spaceship creation

In [5]:
ml_string = lsystem.hl_solver.translator.transform(string=spaceship_string)
ll_solution = lsystem.ll_solver.solve(string=ml_string,
                                   iterations=1,
                                   strings_per_iteration=1,
                                   check_sat=False)[0]
base_position, orientation_forward, orientation_up = Vec.v3i(
            0, 0, 0), Orientation.FORWARD.value, Orientation.UP.value
structure = Structure(origin=base_position,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up)
structure = LLStructureMaker(
    atoms_alphabet=lsystem.ll_solver.atoms_alphabet,
    position=base_position).fill_structure(structure=structure,
                                           string=ll_solution.string)
structure.sanify()
interactive_plot(structure=structure,
                 title='Original structure')

In [6]:
# b = structure._blocks[(50,90,15)]

# print(b.block_type)

# for mp in b.mountpoints:
#     print(f'Normal: {mp.face}\tStart: {mp.start}\tEnd: {mp.end}')

# computed_mps = {}

# for mp in b.mountpoints:
#     valid_mps = []
#     # xrange, yrange, zrange = abs(mp.end.x - mp.start.x), abs(mp.end.y - mp.start.y), abs(mp.end.z - mp.start.z)
#     for x in range(mp.start.x, mp.end.x + 1, 1 if mp.end.x >= mp.start.x else -1):
#         for y in range(mp.start.y, mp.end.y + 1, 1 if mp.end.y >= mp.start.y else -1):
#             for z in range(mp.start.z, mp.end.z + 1, 1 if mp.end.z >= mp.start.z else -1):
#                 valid_mps.append(mp.start.sum(Vec(x, y, z)))
#     computed_mps[mp.face.as_tuple()] = valid_mps

# # print(computed_mps)

# for k, v in computed_mps.items():
#     print(Vec.from_tuple(k), len(v))

In [7]:
# b = structure._blocks[(50,20,15)]

# print(b.block_type)

# for mp in b.mountpoints:
#     print(f'Normal: {mp.face}\tStart: {mp.start}\tEnd: {mp.end}')

# for mp in b.mountpoints:
#     valid_mps = []
#     # xrange, yrange, zrange = abs(mp.end.x - mp.start.x), abs(mp.end.y - mp.start.y), abs(mp.end.z - mp.start.z)
#     for x in range(mp.start.x, mp.end.x + 1, 1 if mp.end.x >= mp.start.x else -1):
#         for y in range(mp.start.y, mp.end.y + 1, 1 if mp.end.y >= mp.start.y else -1):
#             for z in range(mp.start.z, mp.end.z + 1, 1 if mp.end.z >= mp.start.z else -1):
#                 valid_mps.append(mp.start.sum(Vec(x, y, z)))
#     computed_mps[mp.face.as_tuple()] = valid_mps

# # print(computed_mps)

# for k, v in computed_mps.items():
#     print(Vec.from_tuple(k), len(v))

In [8]:
# from pcgsepy.common.vecs import orientation_from_vec, rotate, get_rotation_matrix
# from pcgsepy.structure import Block
# from scipy.spatial.transform import Rotation

# b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
#                        orientation_forward=orientation_from_vec(Vec(0, 0, -1)),
#                        orientation_up=orientation_from_vec(Vec(0, 1, 0)))

# b2 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
#                        orientation_forward=orientation_from_vec(Vec(0, 0, -1)),
#                        orientation_up=orientation_from_vec(Vec(-1, 0, 0)))

# direction = Orientation.DOWN

# rot_direction = rotate(get_rotation_matrix(forward=b1.orientation_forward, up=b1.orientation_up), vector=direction.value)

# mp1 = [mp for mp in b1.mountpoints if mp.face == rot_direction]

# print(f'Block 1: {b1}\nMountpoint 1: {mp1}')

# starts1, ends1 = [], []
# for mp in mp1:
#     # new_start = rotate(get_rotation_matrix(forward=b1.orientation_forward, up=b1.orientation_up), vector=mp.start)
#     # new_end = rotate(get_rotation_matrix(forward=b1.orientation_forward, up=b1.orientation_up), vector=mp.end)
#     new_start = Vec.from_tuple(mp.start.as_tuple())
#     new_end = Vec.from_tuple(mp.end.as_tuple())
#     starts1.append(Vec(x=new_start.x if rot_direction.x == -1 or rot_direction.x == 0 else new_start.x - b1.scaled_size.x,
#                       y=new_start.y if rot_direction.y == -1 or rot_direction.y == 0 else new_start.y - b1.scaled_size.y,
#                       z=new_start.z if rot_direction.z == -1 or rot_direction.z == 0 else new_start.z - b1.scaled_size.z))
#     ends1.append(Vec(x=new_end.x if rot_direction.x == -1 or rot_direction.x == 0 else new_end.x - b1.scaled_size.x,
#                     y=new_end.y if rot_direction.y == -1 or rot_direction.y == 0 else new_end.y - b1.scaled_size.y,
#                     z=new_end.z if rot_direction.z == -1 or rot_direction.z == 0 else new_end.z - b1.scaled_size.z))

# print(f'Corrected mount points: {starts1}, {ends1}')

# opposite = orientation_from_vec(direction.value.opposite())

# # offset = direction.value.scale(structure.grid_size).to_veci()

# # b2 = structure._blocks[b1.position.sum(offset).as_tuple()]

# rot_opposite = rotate(get_rotation_matrix(forward=b2.orientation_forward, up=b2.orientation_up),
#                       vector=opposite.value)

# mp2 = [mp for mp in b2.mountpoints if mp.face == rot_opposite]

# print(f'Block 2: {b2}\nMountpoint 2: {mp2}')

# starts2, ends2 = [], []
# for mp in mp2:
#     new_start = Vec.from_tuple(mp.start.as_tuple())
#     new_end = Vec.from_tuple(mp.end.as_tuple())
#     new_start = Vec(x=new_start.x if rot_opposite.x == -1 or rot_opposite.x == 0 else new_start.x - b2.scaled_size.x,
#                       y=new_start.y if rot_opposite.y == -1 or rot_opposite.y == 0 else new_start.y - b2.scaled_size.y,
#                       z=new_start.z if rot_opposite.z == -1 or rot_opposite.z == 0 else new_start.z - b2.scaled_size.z)
#     new_end = Vec(x=new_end.x if rot_opposite.x == -1 or rot_opposite.x == 0 else new_end.x - b2.scaled_size.x,
#                     y=new_end.y if rot_opposite.y == -1 or rot_opposite.y == 0 else new_end.y - b2.scaled_size.y,
#                     z=new_end.z if rot_opposite.z == -1 or rot_opposite.z == 0 else new_end.z - b2.scaled_size.z)
    
#     print(new_start, new_end)
#     print(Rotation.from_matrix(get_rotation_matrix(forward=b2.orientation_forward, up=b2.orientation_up)).inv().as_matrix())
    
#     new_start = rotate(Rotation.from_matrix(get_rotation_matrix(forward=b2.orientation_forward, up=b2.orientation_up)).inv().as_matrix(), vector=new_start)
#     new_end = rotate(Rotation.from_matrix(get_rotation_matrix(forward=b2.orientation_forward, up=b2.orientation_up)).inv().as_matrix(), vector=new_end)
#     starts2.append(new_start)
#     ends2.append(new_end)

# print(f'Corrected mount points: {starts2}, {ends2}')



# # eos1 = [Vec(x=max(mp.start.x, mp.end.x), y=max(mp.start.y, mp.end.y), z=max(mp.start.z, mp.end.z)) for mp in mp1]
# # sos1 = [Vec(x=min(mp.start.x, mp.end.x), y=min(mp.start.y, mp.end.y), z=min(mp.start.z, mp.end.z)) for mp in mp1]

# # eos2 = [Vec(x=max(mp.start.x, mp.end.x), y=max(mp.start.y, mp.end.y), z=max(mp.start.z, mp.end.z)) for mp in mp2]
# # sos2 = [Vec(x=min(mp.start.x, mp.end.x), y=min(mp.start.y, mp.end.y), z=min(mp.start.z, mp.end.z)) for mp in mp2]

# eos1 = [Vec(x=max(start.x, end.x), y=max(start.y, end.y), z=max(start.z, end.z)) for (start, end) in zip(starts1, ends1)]
# sos1 = [Vec(x=min(start.x, end.x), y=min(start.y, end.y), z=min(start.z, end.z)) for (start, end) in zip(starts1, ends1)]

# eos2 = [Vec(x=max(start.x, end.x), y=max(start.y, end.y), z=max(start.z, end.z)) for (start, end) in zip(starts2, ends2)]
# sos2 = [Vec(x=min(start.x, end.x), y=min(start.y, end.y), z=min(start.z, end.z)) for (start, end) in zip(starts2, ends2)]

# # assume b1 will always be "smaller" than b2
# # then all mp1 should be contained in some mp2
# valid = True
# for eo1, so1 in zip(eos1, sos1):
#         for eo2, so2 in zip(eos2, sos2):
#                 # if so2.x <= so1.x <= eo1.x <= eo2.x and so2.y <= so1.y <= eo1.y <= eo2.y and so2.z <= so1.z <= eo1.z <= eo2.z:
#                 valid &= so1.x >= so2.x and eo1.x <= eo2.x and so1.y >= so2.y and eo1.y <= eo2.y and so1.z >= so2.z and eo1.z <= eo2.z
#                     # valid &= True

# print(f'Is valid? {valid}')


In [9]:
# rotmat = np.asarray([[ 1., -0.,  0.], [ 0.,  1.,  0.], [-0.,  0.,  1.]])
# v = Vec(0,5,0)
# rotate(rotmat, v)

## Hullbuilder

In [10]:
# from typing import Tuple
# from pcgsepy.structure import Block

# def vec_to_idx(v: Vec) -> Tuple[int, int, int]:
#     return (v.x, v.y, v.z)

# def idx_to_vec(idx: Tuple[int, int, int]) -> Vec:
#     return Vec(x=idx[0], y=idx[1], z=idx[2])

# def is_slope_block(block: Block) -> bool:
#     return "Slope" in block.block_type or "Corner" in block.block_type

In [11]:
from itertools import combinations
from scipy.spatial import ConvexHull, Delaunay
from scipy.ndimage import grey_erosion, binary_erosion
import numpy as np
import numpy.typing as npt
from pcgsepy.common.vecs import Orientation, Vec, orientation_from_vec
from pcgsepy.structure import Block, Structure, MountPoint
from typing import List, Optional, Tuple
from itertools import product
from pcgsepy.common.vecs import rotate, get_rotation_matrix
from scipy.spatial.transform import Rotation
from scipy.ndimage import generic_gradient_magnitude, sobel

class HullBuilder:
    def __init__(self,
                 erosion_type: str,
                 apply_erosion: bool,
                 apply_smoothing: bool):
        self.AIR_BLOCK_VALUE = 0
        self.BASE_BLOCK_VALUE = 1
        self.SLOPE_BLOCK_VALUE = 2
        self.CORNER_BLOCK_VALUE = 3
        self.CORNERINV_BLOCK_VALUE = 4
        self.CORNERSQUARE_BLOCK_VALUE = 5
        self.CORNERSQUAREINV_BLOCK_VALUE = 6
        self.available_erosion_types = ['grey', 'bin']
        self.erosion_type = erosion_type
        assert self.erosion_type in self.available_erosion_types, f'Unrecognized erosion type {self.erosion_type}; available are {self.available_erosion_types}.'
        if self.erosion_type == 'grey':
            self.erosion = grey_erosion
            self.footprint=[
                [
                    [False, False, False],
                    [False, True, False],
                    [False, False, False]
                ],
                [
                    [False, True, False],
                    [True, True, True],
                    [False, True, False]
                ],
                [
                    [False, False, False],
                    [False, True, False],
                    [False, False, False]
                ]
            ]
        elif self.erosion_type == 'bin':
            self.erosion = binary_erosion
            self.iterations = 2
        self.apply_erosion = apply_erosion
        self.apply_smoothing = apply_smoothing
        
        self.base_block = 'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock'
        self.obstruction_targets = ['window']
        self._blocks_set = {}
        
        self._orientations = [Orientation.FORWARD, Orientation.BACKWARD, Orientation.UP, Orientation.DOWN, Orientation.LEFT, Orientation.RIGHT]
        self._valid_orientations = [(of, ou) for (of, ou) in list(product(self._orientations, self._orientations)) if of != ou and of != orientation_from_vec(ou.value.opposite())]
        self._smoothing_order = {
            self.BASE_BLOCK_VALUE: [self.SLOPE_BLOCK_VALUE],#, self.CORNERSQUARE_BLOCK_VALUE, self.CORNER_BLOCK_VALUE],
            # self.CORNERSQUAREINV_BLOCK_VALUE: [],
            # self.CORNERINV_BLOCK_VALUE: [],
            # self.SLOPE_BLOCK_VALUE: [self.CORNERSQUARE_BLOCK_VALUE, self.CORNER_BLOCK_VALUE]
            }
        self.block_value_types = {
            self.BASE_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
            self.SLOPE_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
            self.CORNER_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
            self.CORNERINV_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
            self.CORNERSQUARE_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerSquare',
            self.CORNERSQUAREINV_BLOCK_VALUE: 'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerSquareInverted',
        }
    
    def get_structure_iterators(self,
                                structure: Structure) -> Tuple[Vec, List[Vec]]:
        scale = structure.grid_size
        return scale, self._orientations
    
    def _get_convex_hull(self,
                         arr: np.ndarray) -> np.ndarray:
        """Compute the convex hull of the given array.

        Args:
            arr (np.ndarray): The Structure's array.

        Returns:
            np.ndarray: The convex hull.
        """
        points = np.transpose(np.where(arr))
        hull = ConvexHull(points)
        deln = Delaunay(points[hull.vertices])
        idx = np.stack(np.indices(arr.shape), axis=-1)
        out_idx = np.nonzero(deln.find_simplex(idx) + 1)
        out_arr = np.zeros(arr.shape)
        out_arr[out_idx] = self.BASE_BLOCK_VALUE
        # out_arr[np.nonzero(arr)] = arr[np.nonzero(arr)]
        return out_arr
        
    def _adj_to_spaceship(self,
                          i: int,
                          j: int,
                          k: int,
                          spaceship: np.ndarray) -> bool:
        """Check coordinates adjacency to original spaceship hull.

        Args:
            i (int): The i coordinate
            j (int): The j coordiante
            k (int): The k coordinate
            spaceship (np.ndarray): The original spaceship hull

        Returns:
            bool: Whether the coordinate is adjacent to the original spaceship
        """
        adj = False
        for di, dj, dk in zip([+1, 0, 0, 0, 0, -1], [0, +1, 0, 0, -1, 0], [0, 0, +1, -1, 0, 0]):
            if 0 < i + di < spaceship.shape[0] and 0 < j + dj < spaceship.shape[1] and 0 < k + dk < spaceship.shape[2]:
                adj |= spaceship[i + di, j + dj, k + dk] != 0
        return adj

    def _add_block(self,
                   block_type: str,
                   idx: Tuple[int, int, int],
                   pos: Vec,
                   orientation_forward: Orientation = Orientation.FORWARD,
                   orientation_up: Orientation = Orientation.UP) -> None:
        """Add the block to the structure.

        Args:
            block_type (str): The block type.
            structure (Structure): The structure.
            pos (Tuple[int, int, int]): The grid coordinates (non-grid-size specific)
            orientation_forward (Orientation, optional): The forward orientation of the block. Defaults to Orientation.FORWARD.
            orientation_up (Orientation, optional): The up orientation of the block. Defaults to Orientation.UP.
        """
        block = Block(block_type=block_type,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up)
        block.position = pos
        self._blocks_set[idx] = block

    def _tag_internal_air_blocks(self,
                                 arr: np.ndarray):
        air_blocks = np.zeros(shape=arr.shape)
        for i in range(arr.shape[0]):
            for j in range(arr.shape[1]):
                for k in range(arr.shape[2]):
                    if sum(arr[0:i, j, k]) != 0 and \
                        sum(arr[i:arr.shape[0], j, k]) != 0 and \
                        sum(arr[i, 0:j, k]) != 0 and \
                        sum(arr[i, j:arr.shape[1], k]) != 0 and \
                        sum(arr[i, j, 0:k]) != 0 and \
                        sum(arr[i, j, k:arr.shape[2]]) != 0:
                            air_blocks[i, j, k] = self.BASE_BLOCK_VALUE
        return air_blocks
        
    def _exists_block(self,
                      idx: Tuple[int, int, int],
                      structure: Structure) -> bool:
        return structure._blocks.get(idx, None) is not None
    
    def _within_hull(self,
                     loc: Vec,
                     hull: npt.NDArray[np.float32]) -> bool:
        i, j, k = loc.as_tuple()
        return 0 <= i < hull.shape[0] and 0 <= j < hull.shape[1] and 0 <= k < hull.shape[2]
    
    def _is_air_block(self,
                      loc: Vec,
                      structure: Structure,
                      hull: np.typing.NDArray) -> bool:
        return not self._exists_block(idx=loc.as_tuple(), structure=structure) and\
            (self._within_hull(loc=loc.scale(1 / structure.grid_size).to_veci(), hull=hull) and hull[loc.scale(1 / structure.grid_size).to_veci().as_tuple()] == self.AIR_BLOCK_VALUE)
    
    def _next_to_target(self,
                        loc: Vec,
                        structure: Structure,
                        direction: Vec) -> bool:
        dloc = loc.sum(direction)
        if self._exists_block(idx=dloc.as_tuple(), structure=structure):
            obs_block = structure._blocks.get(dloc.as_tuple())
            return any([target.lower() in obs_block.block_type.lower() for target in self.obstruction_targets])
    
    def _remove_in_direction(self,
                             loc: Vec,
                             hull: npt.NDArray[np.float32],
                             direction: Vec) -> npt.NDArray[np.float32]:
        i, j, k = loc.as_tuple()
        di, dj, dk = direction.as_tuple()
        while 0 < i < hull.shape[0] and 0 < j < hull.shape[1] and 0 < k < hull.shape[2]:
            hull[i, j, k] = self.AIR_BLOCK_VALUE
            i += di
            j += dj
            k += dk
        return hull
    
    def _remove_obstructing_blocks(self,
                                   hull: npt.NDArray[np.float32],
                                   structure: Structure) -> npt.NDArray[np.float32]:
        ii, jj, kk = hull.shape
        for i in range(ii):
            for j in range(jj):
                for k in range(kk):
                    if hull[i, j, k] != self.AIR_BLOCK_VALUE:
                        scale = structure.grid_size
                        loc = Vec.from_tuple((scale * i, scale * j, scale * k))
                        for direction in self._orientations:                            
                            if self._next_to_target(loc=loc,
                                                    structure=structure,
                                                    direction=direction.value.scale(scale)):
                                hull = self._remove_in_direction(loc=loc.scale(v=1 / structure.grid_size).to_veci(),
                                                                 hull=hull,
                                                                 direction=direction.value.opposite())
        return hull
    
    def _get_outer_indices(self,
                           arr: npt.NDArray[np.float32],
                           edges_only: bool = False,
                           corners_only: bool = False) -> Tuple[npt.NDArray[np.int64], npt.NDArray[np.int64], npt.NDArray[np.int64]]:
        # __threshold = 1
        # mag = generic_gradient_magnitude(arr, sobel, mode='nearest', cval=self.AIR_BLOCK_VALUE)
        # mag[arr == self.AIR_BLOCK_VALUE] = self.AIR_BLOCK_VALUE
        # return np.where(mag >= (np.max(mag) * __threshold))
        idxs_i, idxs_j, idxs_k = [], [], []
        for direction in self._orientations:
            di, dj, dk = direction.value.x, direction.value.y, direction.value.z
            
            if di != 0:
                face = np.ones(shape=(arr.shape[1], arr.shape[2])) * self.AIR_BLOCK_VALUE
                i = 0 if di >= 0 else -1
                while -arr.shape[0] < i < arr.shape[0]:
                    for j in range(face.shape[0]):
                        for k in range(face.shape[1]):
                            if face[j, k] == self.AIR_BLOCK_VALUE and arr[i, j, k] != self.AIR_BLOCK_VALUE:
                                face[j, k] = 1 if self.AIR_BLOCK_VALUE != 1 else -self.AIR_BLOCK_VALUE
                                idxs_i.append((i, j, k) if i >= 0 else (arr.shape[0] + i, j, k))
                    i += di
                
            elif dj != 0:
                face = np.ones(shape=(arr.shape[0], arr.shape[2])) * self.AIR_BLOCK_VALUE
                
                j = 0 if dj >= 0 else -1
                while -arr.shape[1] < j < arr.shape[1]:
                    for i in range(face.shape[0]):
                        for k in range(face.shape[1]):
                            if face[i, k] == self.AIR_BLOCK_VALUE and arr[i, j, k] != self.AIR_BLOCK_VALUE:
                                face[i, k] = 1 if self.AIR_BLOCK_VALUE != 1 else -self.AIR_BLOCK_VALUE
                                idxs_j.append((i, j, k) if j >= 0 else (i, arr.shape[1] + j, k))
                    j += dj
                                
            else:
                face = np.ones(shape=(arr.shape[0], arr.shape[1])) * self.AIR_BLOCK_VALUE
                
                k = 0 if dk >= 0 else -1
                while -arr.shape[2] < k < arr.shape[2]:
                    for i in range(face.shape[0]):
                        for j in range(face.shape[1]):
                            if face[i, j] == self.AIR_BLOCK_VALUE and arr[i, j, k] != self.AIR_BLOCK_VALUE:
                                face[i, j] = 1 if self.AIR_BLOCK_VALUE != 1 else -self.AIR_BLOCK_VALUE
                                idxs_k.append((i, j, k) if k >= 0 else (i, j, arr.shape[2] + k))
                    k += dk
        if corners_only:
            indices = list(set(idxs_i).intersection(set(idxs_j)).intersection(set(idxs_k)))
        elif edges_only:
            indices =  list(set(idxs_i).intersection(set(idxs_j))) + list(set(idxs_j).intersection(set(idxs_k))) + list(set(idxs_j).intersection(set(idxs_k)))
        else:
            indices =  list(set(idxs_i + idxs_j + idxs_k))
        indices = list(set(indices))
        return np.asarray([idx[0] for idx in indices]), np.asarray([idx[1] for idx in indices]), np.asarray([idx[2] for idx in indices])
    
    def try_and_get_block(self,
                          idx: Tuple[int, int, int],
                          offset: Vec,
                          structure: Structure) -> Optional[Block]:
        """Try and get a block from either the hull or the structure.

        Args:
            idx (Tuple[int, int, int]): The index of the block (hull-scaled).
            offset (Vec): The offset vector (structure-scaled).
            structure (Structure): The structure.

        Returns:
            Optional[Block]: The block at `idx + offset`, if it exists.
        """
        dloc = Vec.from_tuple(idx).scale(structure.grid_size).sum(offset)
        if self._exists_block(idx=dloc.as_tuple(),
                              structure=structure):
            return structure._blocks[dloc.as_tuple()]
        else:
            return self._blocks_set.get(Vec.from_tuple(idx).sum(offset.scale(1 / structure.grid_size)).to_veci().as_tuple(), None)
    
    def _get_mountpoint_limits(self,
                               mountpoints: List[MountPoint],
                               block: Block,
                               rot_direction: Vec,
                               realign: bool = False) -> Tuple[List[Vec], List[Vec], List[int]]:
        starts, ends = [], []
        for mp in mountpoints:
            # realigned_start = Vec(x=mp.start.x if rot_direction.x == -1 or rot_direction.x == 0 else mp.start.x - block.scaled_size.x,
            #                       y=mp.start.y if rot_direction.y == -1 or rot_direction.y == 0 else mp.start.y - block.scaled_size.y,
            #                       z=mp.start.z if rot_direction.z == -1 or rot_direction.z == 0 else mp.start.z - block.scaled_size.z)
            # realigned_end = Vec(x=mp.end.x if rot_direction.x == -1 or rot_direction.x == 0 else mp.end.x - block.scaled_size.x,
            #                     y=mp.end.y if rot_direction.y == -1 or rot_direction.y == 0 else mp.end.y - block.scaled_size.y,
            #                     z=mp.end.z if rot_direction.z == -1 or rot_direction.z == 0 else mp.end.z - block.scaled_size.z)
            realigned_start = mp.start
            realigned_end = mp.end
            
            if realign:
                inv_rot_matrix = Rotation.from_matrix(get_rotation_matrix(forward=block.orientation_forward,
                                                                        up=block.orientation_up)).inv().as_matrix()
                
                realigned_start = rotate(rotation_matrix=inv_rot_matrix,
                                        vector=realigned_start)
                realigned_end = rotate(rotation_matrix=inv_rot_matrix,
                                    vector=realigned_end)
                
            starts.append(realigned_start)
            ends.append(realigned_end)
            
        ordered_ends = [Vec(x=max(start.x, end.x), y=max(start.y, end.y), z=max(start.z, end.z)) for (start, end) in zip(starts, ends)]
        ordered_starts = [Vec(x=min(start.x, end.x), y=min(start.y, end.y), z=min(start.z, end.z)) for (start, end) in zip(starts, ends)]
        
        planes = [end.diff(start).vol(ignore_zero=False) for start, end in zip(ordered_starts, ordered_ends)]
        
        return ordered_starts, ordered_ends, planes
    
    def _check_valid_placement(self,
                               idx: Tuple[int, int, int],
                               block: Block,
                               direction: Orientation,
                               hull: np.typing.NDArray,
                               structure: Structure) -> Tuple[bool, int]:
        """Check if the block could be placed with the given orientation when checking in the specified direction.

        Args:
            idx (Tuple[int, int, int]): The index of the block (hull-scaled).
            block (Block): The block to be checked.
            direction (Orientation): The direction to check placement for.
            hull (np.typing.NDArray): The hull.
            structure (Structure): The structure.

        Returns:
            bool: Whether the block could be placed with the given orientation when checking in the specified direction.
        """
        rot_direction = rotate(get_rotation_matrix(forward=block.orientation_forward,
                                                   up=block.orientation_up),
                               vector=direction.value).to_veci()
        mp1 = [mp for mp in block.mountpoints if mp.face == rot_direction]
        starts1, ends1, planes1 = self._get_mountpoint_limits(mountpoints=mp1,
                                                              block=block,
                                                              rot_direction=rot_direction,
                                                              realign=True)
        other_block = self.try_and_get_block(idx=idx,
                                             offset=direction.value.scale(structure.grid_size).to_veci(),
                                             structure=structure)
        if other_block is None:
            # facing air block, can always be placed
            if mp1 != []:
                
                # print(block, other_block, '\t', starts1, ends1, '\t', 'AIR BLOCK', '\t', True)
                
                return True, sum(planes1)
            else:
                face = Vec(x=block.scaled_size.x if rot_direction.x == 0 else 0,
                           y=block.scaled_size.y if rot_direction.y == 0 else 0,
                           z=block.scaled_size.z if rot_direction.z == 0 else 0)
                
                # print(block, other_block, '\t', 'NO MOUNTPOINTS', '\t', 'AIR BLOCK', '\t', False)
                
                return True, face.vol(ignore_zero=False)
        else:
            if mp1 == []:
                # print(f'No mountpoints in {direction} ({rot_direction}) for {block}')
                
                # print(block, other_block, '\t', 'NO MOUNTPOINTS', '\t', '--', '\t', False)
                
                return False, 0
        
        opposite_direction = orientation_from_vec(direction.value.opposite())
        rot_other = rotate(get_rotation_matrix(forward=other_block.orientation_forward,
                                               up=other_block.orientation_up),
                           vector=opposite_direction.value).to_veci()
                
        mp2 = [mp for mp in other_block.mountpoints if mp.face == rot_other]
        if mp2 == []:
            # print(f'No mountpoints in {opposite_direction} ({rot_other}) for {other_block}')
                
            # print(block, other_block, '\t', '--', '\t', 'NO MOUNTPOINTS', '\t', False)
                
            return False, 0
        starts2, ends2, planes2 = self._get_mountpoint_limits(mountpoints=mp2,
                                                              block=other_block,
                                                              rot_direction=rot_other,
                                                              realign=True)
        
        # assume block will always be "smaller" than other_block
        # then all mp1 should be contained in some mp2
        all_valid = []
        coverage_err = 0
        for eo1, so1, p1 in zip(ends1, starts1, planes1):
            assert p1 != 0, f'Mountpoint with empty surface: {mp1} has {so1}-{eo1} (from block {block})'
            mp_valid = True
            for eo2, so2, p2 in zip(ends2, starts2, planes2):
                assert p2 != 0, f'Mountpoint with empty surface: {mp2} has {so2}-{eo2} (from block {other_block})'
                # NOTE: This check does not take into account exclusions and properties masks (yet)
                # Sources\VRage.Math\BoundingBoxI.cs#328
                # (double)this.Max.X >= (double)box.Min.X && (double)this.Min.X <= (double)box.Max.X && ((double)this.Max.Y >= (double)box.Min.Y && (double)this.Min.Y <= (double)box.Max.Y) && ((double)this.Max.Z >= (double)box.Min.Z && (double)this.Min.Z <= (double)box.Max.Z);
                mp_valid &= so1.x >= so2.x and eo1.x <= eo2.x and so1.y >= so2.y and eo1.y <= eo2.y and so1.z >= so2.z and eo1.z <= eo2.z
                # compute coverage error for the mountpoint
                coverage_err += abs(p2 - p1)
            all_valid.append(mp_valid)
        
        # print(block, other_block, '\t', starts1, ends1, '\t', starts2, ends2, '\t', all_valid)
        
        return all(all_valid), coverage_err
        
    def _check_valid_position(self,
                              idx: Tuple[int, int, int],
                              block: Block,
                              hull: np.typing.NDArray,
                              structure: Structure) -> Tuple[bool, int]:
        valid = True
        area_err = 0
        for direction in self._orientations:
            res, delta_area = self._check_valid_placement(idx=idx,
                                                          block=block,
                                                          direction=direction,
                                                          hull=hull,
                                                          structure=structure)
            valid &= res
            area_err += delta_area
            if not valid:
                break
            
        return valid, delta_area
    
    def try_smoothing(self,
                      idx: Tuple[int, int, int],
                      hull: np.typing.NDArray,
                      structure: Structure) -> Optional[Block]:
        i, j, k = idx
        scale = structure.grid_size
        loc = Vec.from_tuple((scale * i, scale * j, scale * k))
        
        block_type = hull[i, j, k]
        block = self._blocks_set[idx]
        
        # removal check
        valid, _ = self._check_valid_position(idx=idx,
                                              block=block,
                                              hull=hull,
                                              structure=structure)
        if not valid:
            return None, self.AIR_BLOCK_VALUE
        # replacement check   
        elif block_type in self._smoothing_order.keys():
            for possible_type in self._smoothing_order[block_type]:
                orientation_scores, valids = np.zeros(shape=len(self._valid_orientations), dtype=np.int64), np.zeros(shape=len(self._valid_orientations), dtype=np.bool8)
                for i, (of, ou) in enumerate(self._valid_orientations):
                    possible_block = Block(block_type=self.block_value_types[possible_type],
                                           orientation_forward=of,
                                           orientation_up=ou)
                    valid, err = self._check_valid_position(idx=idx,
                                                  block=possible_block,
                                                  hull=hull,
                                                  structure=structure)
                    orientation_scores[i] = err if valid else 9999
                    valids[i] = valid
                
                if idx == (1, 18, 3):
                    print(orientation_scores, valids)
                
                if any(valids):
                    of, ou = self._valid_orientations[np.argmin(orientation_scores)]
                    return Block(block_type=self.block_value_types[possible_type],
                                 orientation_forward=of,
                                 orientation_up=ou), possible_type
            return None, block_type
        # skip
        else:
            return None, block_type
        
    def _get_ordered_idxs(self,
                          arr: npt.NDArray[np.float32]) -> List[Tuple[int, int, int]]:
        xxs, yys, zzs = [], [], []
        edges = self._get_outer_indices(arr=arr,
                                        edges_only=True)
        arr_copy = np.zeros_like(arr)
        arr_copy[edges] = 1
        while np.sum(arr_copy.flatten()) != 0:
            idxs = self._get_outer_indices(arr=arr_copy,
                                        corners_only=True)
            xxs.extend(idxs[0].tolist())
            yys.extend(idxs[1].tolist())
            zzs.extend(idxs[2].tolist())
            arr_copy[idxs] = 0
            
        return [(x, y, z) for x, y, z in zip(xxs, yys, zzs)]
           
    def add_external_hull(self,
                          structure: Structure) -> None:
        """Add an external hull to the given Structure.
        This process adds the hull blocks directly into the Structure, so it can be used only once per spaceship.

        Args:
            structure (Structure): The spaceship.
        """
        arr = structure.as_grid_array()
        air = self._tag_internal_air_blocks(arr=arr)
        hull = self._get_convex_hull(arr=arr)
        hull[np.nonzero(air)] = self.AIR_BLOCK_VALUE
        hull[np.nonzero(arr)] = self.AIR_BLOCK_VALUE
        
        if self.apply_erosion:
            if self.erosion_type == 'grey':
                hull = grey_erosion(input=hull,
                                    footprint=self.footprint,
                                    mode='constant',
                                    cval=1)
                hull = hull.astype(int)
                hull *= self.BASE_BLOCK_VALUE                
            elif self.erosion_type == 'bin':
                mask = np.zeros(arr.shape)
                for i in range(mask.shape[0]):
                    for j in range(mask.shape[1]):
                        for k in range(mask.shape[2]):
                            mask[i, j, k] = self.AIR_BLOCK_VALUE if self._adj_to_spaceship(i=i, j=j, k=k, spaceship=arr) else self.BASE_BLOCK_VALUE
                hull = binary_erosion(input=hull,
                                      mask=mask,
                                      iterations=self.iterations)
                hull = hull.astype(int)
                hull *= self.BASE_BLOCK_VALUE
        
        # remove all blocks that obstruct target block type
        hull = self._remove_obstructing_blocks(hull=hull,
                                               structure=structure)
        
        # add blocks to self._blocks_set
        for i in range(hull.shape[0]):
                for j in range(hull.shape[1]):
                    for k in range(hull.shape[2]):
                        if hull[i, j, k] != self.AIR_BLOCK_VALUE:
                            self._add_block(block_type=self.base_block,
                                            idx=(i, j, k),
                                            pos=Vec.v3i(i, j, k).scale(v=structure.grid_size),
                                            orientation_forward=Orientation.FORWARD,
                                            orientation_up=Orientation.UP)
        
        # initial blocks removal check
        for i in range(hull.shape[0]):
                for j in range(hull.shape[1]):
                    for k in range(hull.shape[2]):
                        if hull[i, j, k] != self.AIR_BLOCK_VALUE:
                            block = self._blocks_set[(i, j, k)]
                            valid, _ = self._check_valid_position(idx=(i, j, k),
                                                                    block=block,
                                                                    hull=hull,
                                                                    structure=structure)
                            # TODO: Some blocks are not removed even though their symmetrical counterpart are.
                            # It seems to be a problem with the orientations of the blocks in the spaceship.
                            # Example: (9, 8, 1) and (9, 8, 7), both have a LargeBlockArmorCornerInv on the RIGHT
                            # but only one of them is removed correctly (due to mountpoints).
                            if not valid:
                                hull[i, j, k] = self.AIR_BLOCK_VALUE
                                self._blocks_set.pop((i, j, k))
        
        if self.apply_smoothing:
            modified = 1
            while modified != 0:
                modified = 0
                to_rem = []
                for (i, j, k) in self._get_ordered_idxs(arr=hull):
                    block = self._blocks_set[(i, j, k)]
                    # print((i, j, k), block)
                    
                    if (i, j, k) == (1, 18, 3) or (i, j, k) == (1, 18, 4) or (i, j, k) == (1, 18, 5):
                        print((i, j, k))
                    
                    substitute_block, val = self.try_smoothing(idx=(i, j, k),
                                                               hull=hull,
                                                               structure=structure)
                    # print('->', substitute_block, val)
                    if (i, j, k) == (1, 18, 3) or (i, j, k) == (1, 18, 4) or (i, j, k) == (1, 18, 5):
                        print((i, j, k), ' -> ', substitute_block, val)
                    if substitute_block is not None:
                        substitute_block.position = block.position
                        self._blocks_set[(i, j, k)] = substitute_block
                        modified += 1
                    elif substitute_block is None and val == self.AIR_BLOCK_VALUE:
                        to_rem.append((i, j, k))
                        modified += 1
                    hull[i, j, k] = val
                for r in to_rem:
                    self._blocks_set.pop(r)
                # break
        
        # add blocks to structure
        for k, block in self._blocks_set.items():
            structure.add_block(block=block,
                                grid_position=block.position.as_tuple())

In [12]:
hullbuilder = HullBuilder(erosion_type='bin', apply_erosion=True, apply_smoothing=True)

hullbuilder.add_external_hull(structure=structure)

interactive_plot(structure=structure,
                 title='Structure w/ hull (smoothed)')

IndexError: arrays used as indices must be of integer (or boolean) type

## In-game placement

In [ ]:
place_ingame = True

In [ ]:
if place_ingame:
    base_position, orientation_forward, orientation_up = get_base_values()
    # place_structure(structure=structure,
    #                 position=base_position,
    #                 orientation_forward=orientation_forward,
    #                 orientation_up=orientation_up,
    #                 batchify=False)
    structure.update(
        origin=Vec.v3f(0., 0., 100.),
        # orientation_forward=orientation_forward,
        orientation_forward=Orientation.FORWARD,
        # orientation_up=orientation_up,
        orientation_up=Orientation.UP,
    )
    toggle_gamemode(GameMode.PLACING)
    all_blocks = structure.get_all_blocks(to_place=True)
    for block in all_blocks:
        block.position = block.position.sum(base_position)
    place_blocks(all_blocks, sequential=False)
    toggle_gamemode(GameMode.EVALUATING)

In [ ]:

# arr = structure.as_grid_array()

# xxs, yys, zzs = [], [], []
# edges = hullbuilder._get_outer_indices(arr=arr,
#                                 edges_only=True)
# arr_copy = np.zeros_like(arr)
# arr_copy[edges] = 1

# arr_color = np.zeros_like(arr)
# i = 1

# while np.sum(arr_copy.flatten()) != 0:
#     idxs = hullbuilder._get_outer_indices(arr=arr_copy,
#                                           corners_only=True)
#     xxs.extend(idxs[0].tolist())
#     yys.extend(idxs[1].tolist())
#     zzs.extend(idxs[2].tolist())
#     arr_copy[idxs] = 0
    
#     arr_color[idxs] = i
#     i += 1
    
# x, y, z = np.nonzero(arr_color)
# cs = [arr_color[i, j, k] for i, j, k in zip(x, y, z)]
# ss = [structure._clean_label(structure.ks[v - 1]) for v in cs]
# fig = px.scatter_3d(x=x,
#                     y=y,
#                     z=z,
#                     color=cs,
#                     # color_discrete_map=_get_colour_mapping(ss),
#                     labels={
#                         'x': 'x',
#                         'y': 'y',
#                         'z': 'z',
#                         'color': 'Block type'
#                     },
#                     title='')

# fig.update_traces(marker=dict(size=4,
#                             line=dict(width=3,
#                                     color='DarkSlateGrey')),
#                     selector=dict(mode='markers'))

# camera = dict(
#     up=dict(x=0, y=0, z=1),
#     center=dict(x=0, y=0, z=0),
#     eye=dict(x=2, y=2, z=2)
#     )

# fig.update_layout(scene=dict(aspectmode='data'),
#                     scene_camera=camera,
#                     paper_bgcolor='rgba(0,0,0,0)',
#                     plot_bgcolor='rgba(0,0,0,0)')

# fig.show()

In [ ]:
# from itertools import product
# from pcgsepy.structure import Block

# orientations = [Orientation.FORWARD, Orientation.BACKWARD, Orientation.UP, Orientation.DOWN, Orientation.LEFT, Orientation.RIGHT]

# base_position, orientation_forward, orientation_up = get_base_values()

# for i, (of, ou) in enumerate(product(orientations, orientations)):
    
#     print(f'{i}: of: {of.name}; ou: {ou.name}')

#     b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
#             orientation_forward=of,
#             orientation_up=ou)
#     b1.position = Vec.v3i(x=(i * 5), y=0, z=0)


#     b1.position = b1.position.sum(base_position)

#     toggle_gamemode(GameMode.PLACING)
#     place_blocks([b1], sequential=False)
#     toggle_gamemode(GameMode.EVALUATING)


In [ ]:
# base_position, orientation_forward, orientation_up = get_base_values()

# dd, du = Orientation.DOWN, Orientation.UP
# dr, dl = Orientation.RIGHT, Orientation.LEFT
# df, db = Orientation.FORWARD, Orientation.BACKWARD

# # for i, (d1, d2, d3, d4) in enumerate(zip([dd, dd, du, du, df, df, df, df, db, db, db, db],
# #                                          [dl, dr, dl, dr, du, dd, dr, dl, du, dd, dr, dl],
# #                                          [du, du, dd, dd, db, db, db, db, df, df, df, df],
# #                                          [dr, dl, dr, dl, dd, du, dl, dr, dd, du, dl, dr])):
# #     b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
# #                orientation_forward=Orientation.FORWARD,
# #                orientation_up=Orientation.UP)
# #     b1.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position).sum(d3.value.scale(5))
# #     b2 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
# #                orientation_forward=Orientation.FORWARD,
# #                orientation_up=Orientation.UP)
# #     b2.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position).sum(d4.value.scale(5))
# #     b3 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
# #                orientation_forward=Orientation.FORWARD,
# #                orientation_up=Orientation.UP)
# #     b3.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position).sum(d4.value.scale(5))
    
# #     slope = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
# #                   orientation_forward=d1,
# #                   orientation_up=d4)
# #     slope.position = Vec.v3f(20 + i * 10, 0, 0).sum(base_position)
    
# #     print(b1, slope, b2)

# b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=Orientation.FORWARD,
#             orientation_up=Orientation.DOWN)
# b1.position = Vec.v3f(0,0,0).sum(base_position)

# b2 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=Orientation.UP,
#             orientation_up=Orientation.FORWARD)
# b2.position = Vec.v3f(0, 0, -5).sum(base_position)
# # b2.position = Vec.v3f(5, 0, 0).sum(base_position)
# # b3 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
# #             orientation_forward=Orientation.FORWARD,
# #             orientation_up=Orientation.LEFT)
# # b3.position = Vec.v3f(0, -5, 0).sum(base_position)


# # b4 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
# #             orientation_forward=Orientation.FORWARD,
# #             orientation_up=Orientation.UP)
# # b4.position = Vec.v3f(0, 0, 0).sum(base_position)

# b1 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=Orientation.LEFT,
#             orientation_up=Orientation.UP)
# b1.position = Vec.v3f(0, 0, -5).sum(base_position)
# b2 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=Orientation.FORWARD,
#             orientation_up=Orientation.UP)
# b2.position = Vec.v3f(-5, 0, 0).sum(base_position)
# b3 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
#             orientation_forward=Orientation.FORWARD,
#             orientation_up=Orientation.RIGHT)
# b3.position = Vec.v3f(0, 5, 0).sum(base_position)

# b4 = Block(block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
#             orientation_forward=Orientation.LEFT,
#             orientation_up=Orientation.UP)
# b4.position = Vec.v3f(0, 0, 0).sum(base_position)

# toggle_gamemode(GameMode.PLACING)
# place_blocks([b1, b2], sequential=False)
# toggle_gamemode(GameMode.EVALUATING)

In [ ]:
# from pcgsepy.common.api_call import call_api, generate_json, compactify_jsons

# res = call_api(jsons=[generate_json(method="Observer.ObserveBlocks")])

# print(res)

In [ ]:
# from pcgsepy.common.api_call import call_api, generate_json, compactify_jsons

# jsons = [generate_json(method="Definitions.BlockDefinitions")]
# res = call_api(jsons=jsons)[0]
# block_definitions = {}
# for v in res['result']:
#     if v['DefinitionId']['Type'] == 'LargeBlockArmorBlock':
#         print(v)

In [ ]:
# mountpoints_slope = []
# mountpoints_corner = []
# mountpoints_cube = []

# for v in res['result']:
#     if v['DefinitionId']['Type'] == 'LargeBlockArmorBlock' or v['DefinitionId']['Type'] == 'LargeBlockArmorSlope' or v['DefinitionId']['Type'] == 'LargeBlockArmorCorner':
#         mountpoints = v['MountPoints']
#         for mp in mountpoints:
#             if v['DefinitionId']['Type'] == 'LargeBlockArmorBlock':
#                 print(mp)
#                 mountpoints_cube.append([Vec.from_json(mp['Start']).as_array(), Vec.from_json(mp['End']).as_array()])
#             if v['DefinitionId']['Type'] == 'LargeBlockArmorSlope':
#                 mountpoints_slope.append([Vec.from_json(mp['Start']).as_array(), Vec.from_json(mp['End']).as_array()])
#             if v['DefinitionId']['Type'] == 'LargeBlockArmorCorner':
#                 mountpoints_corner.append([Vec.from_json(mp['Start']).as_array(), Vec.from_json(mp['End']).as_array()])


In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# xs, ys, zs = [], [], []

# for vec in mountpoints_slope:
#     xs.append(vec[0][0])
#     xs.append(vec[1][0])
    
#     ys.append(vec[0][1])
#     ys.append(vec[1][1])
    
#     zs.append(vec[0][2])
#     zs.append(vec[1][2])
    
    
# fig = px.line_3d(x=xs,
#                  y=ys,
#                  z=zs)

# fig.show()

# fig = px.scatter_3d(x=xs,
#                     y=ys,
#                     z=zs)

# fig.show()

# # fig = plt.figure()
# # ax  = fig.add_subplot(111, projection = '3d')

# # for vec in mountpoints_slope:
# #     ax.plot(vec[0], vec[1], color='b')
    
# # plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# xs, ys, zs = [], [], []

# for vec in mountpoints_corner:
#     xs.append(vec[0][0])
#     xs.append(vec[1][0])
    
#     ys.append(vec[0][1])
#     ys.append(vec[1][1])
    
#     zs.append(vec[0][2])
#     zs.append(vec[1][2])
    
    
# fig = px.line_3d(x=xs,
#                  y=ys,
#                  z=zs)

# fig.show()

# fig = px.scatter_3d(x=xs,
#                     y=ys,
#                     z=zs)

# fig.show()

In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# xs, ys, zs = [], [], []

# for vec in mountpoints_cube:
#     xs.append(vec[0][0])
#     xs.append(vec[1][0])
    
#     ys.append(vec[0][1])
#     ys.append(vec[1][1])
    
#     zs.append(vec[0][2])
#     zs.append(vec[1][2])
    
    
# fig = px.line_3d(x=xs,
#                  y=ys,
#                  z=zs)

# fig.show()

# fig = px.scatter_3d(x=xs,
#                     y=ys,
#                     z=zs)

# fig.show()